In [2]:
import sqlite3
import pandas as pd

In [3]:
# Step 1: Connect to the SQLite database
conn = sqlite3.connect('collection.anki21')
cursor = conn.cursor()

In [5]:
# Step 2: Read the table into a pandas DataFrame
table_name = 'notes'  # Replace with your table name
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

print(df)

Original table:
                 id        guid            mid         mod    usn  \
0     1599309962946  yZ]_uA{Ym;  1596992551063  1599309962      0   
1     1599309962947  E8!Jj>_YCA  1596992551063  1599309962      0   
2     1599309962948  Dh1`k,(}p:  1596992551063  1691884713  16379   
3     1599309962949   SvP|N2jj{  1596992551063  1599309962      0   
4     1599309962950  MboU+s-]mI  1596992551063  1599309962      0   
...             ...         ...            ...         ...    ...   
4241  1599309967187  i2r9to&aSH  1596992551063  1599309963      0   
4242  1599309967188  f2[>,5R8wG  1596992551063  1599309963      0   
4243  1599309967189  pF}#+/s[Ov  1596992551063  1599309963      0   
4244  1599309967190  tK=v<JTxH}  1596992551063  1599309963      0   
4245  1599309967191  eVJ)oK+S2v  1596992551063  1599309963      0   

                                                   tags  \
0                                          orientation    
1                                    

In [8]:
# remove orientation
print(len(df.index))
df = df.loc[~df.tags.str.contains('orientation')]
print(len(df.index))

4246
4239


In [9]:
df.head()

,id,guid,mid,mod,usn,tags,flds,sfld,csum,flags,data
7,1599309962953,iecN7n:djQ,1596992551063,1599309962,0,ends_in_er extreme_irregularity infinitivo ir...,c2e83147-c153-44b4-b453-f77f731ce3bc El verbo ...,El verbo enél eses {{c1::ser}},374306676,0,
8,1599309962954,t1||z{8OP4,1596992551063,1599310148,0,ends_in_er extreme_irregularity gerundio irre...,28974bf1-890e-41fa-bd73-45a7abdfa4cf Mientras ...,Mientras estoy estudiando…ella está {{c1::sien...,4184801480,0,
9,1599309962955,"PhbeK,#2h}",1596992551063,1599309962,0,ends_in_er extreme_irregularity irregular_ver...,07240d22-3470-46fc-8277-3523d9742561 Hugo ha <...,Hugo ha {{c1::sido::…ser…}} (vigilado),851395593,0,
10,1599309962956,m05T}gc0C;,1596992551063,1599309962,0,ends_in_er extreme_irregularity irregular_for...,610fe216-7c31-414b-91fb-19192740ded1 ⊙ Ahora m...,"⊙ Ahora mismo, ⊙yo {{c1::soy::…ser…}} (conscie...",371250322,0,
11,1599309962957,|wzu@j5%J,1596992551063,1599309962,0,dicho ends_in_er extreme_irregularity in_the_...,1c3512c1-80fd-4880-852d-6fa175344917 ⊙ Ahora m...,"⊙ Ahora mismo, ⊙tú {{c1::eres::…ser…}} (consci...",4031152435,0,


In [ ]:

# # Step 3: Modify the DataFrame
# # Example: Add a new column or make any modifications
# df['new_column'] = df['existing_column'] * 2  # Replace with your logic

# print("Modified table:")
# print(df)

# # Step 4: Replace the table in the database
# # First, drop the existing table (if it exists)
# cur = conn.cursor()
# cur.execute(f"DROP TABLE IF EXISTS {table_name}")

# # Then, write the modified DataFrame back to the database
# df.to_sql(table_name, conn, index=False, if_exists='replace')  # Replace mode

# # Step 5: Verify the changes (optional)
# df_updated = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
# print("Updated table in the database:")
# print(df_updated)

# # Step 6: Close the database connection
# conn.close()